In [11]:
from binance.client import Client
from datetime import datetime
import pandas as pd
import os
from dotenv import load_dotenv

class DataCollector:
    def __init__(self):
        load_dotenv()
        self.client = Client(
            os.getenv('BINANCE_API_KEY'),
            os.getenv('BINANCE_API_SECRET')
        )
        
    def get_historical_klines(self, symbol: str, interval: str, start_str: str, end_str: str = None) -> pd.DataFrame:
        """
        Récupère les données historiques pour un symbole donné
        
        Args:
            symbol (str): Paire de trading (ex: 'BTCUSDT')
            interval (str): Intervalle de temps (ex: '1h', '4h', '1d')
            start_str (str): Date de début
            end_str (str, optional): Date de fin. Defaults to None.
            
        Returns:
            pd.DataFrame: DataFrame contenant les données OHLCV
        """
        klines = self.client.get_historical_klines(
            symbol=symbol,
            interval=interval,
            start_str=start_str
        )
        
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        
        # Conversion des colonnes en float
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
            
        return df
    
    def get_current_price(self, symbol: str) -> float:
        """
        Récupère le prix actuel d'un symbole
        
        Args:
            symbol (str): Paire de trading (ex: 'BTCUSDT')
            
        Returns:
            float: Prix actuel
        """
        ticker = self.client.get_symbol_ticker(symbol=symbol)
        return float(ticker['price'])
    
    def get_market_depth(self, symbol: str, limit: int = 100) -> dict:
        """
        Récupère la profondeur du marché pour un symbole
        
        Args:
            symbol (str): Paire de trading (ex: 'BTCUSDT')
            limit (int, optional): Nombre d'ordres à récupérer. Defaults to 100.
            
        Returns:
            dict: Dictionnaire contenant les ordres d'achat et de vente
        """
        depth = self.client.get_order_book(symbol=symbol, limit=limit)
        return {
            'bids': depth['bids'],
            'asks': depth['asks']
        } 
    
DataCollector = DataCollector()

In [14]:
data = DataCollector.get_current_price(symbol='BTCUSDT')
data


109999.99

In [ ]:
data = DataCollector.get_historical_klines(symbol='BTCUSDT', interval='1d', start_str='1 Jan, 2025')
data

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2025-01-01,93576.00,95151.15,92888.00,94591.79,10373.326130,1735775999999,975444194.13799830,1516556,5347.73648000,502914035.64059070,0
2025-01-02,94591.78,97839.50,94392.00,96984.79,21970.489480,1735862399999,2118411852.68127950,3569079,10915.96986000,1052226710.33367170,0
2025-01-03,96984.79,98976.91,96100.01,98174.18,15253.829360,1735948799999,1486610768.59079170,2851164,7468.60797000,727787246.89655880,0
2025-01-04,98174.17,98778.43,97514.79,98220.50,8990.056510,1736035199999,881223751.63536030,1559982,4350.80221000,426524714.13055390,0
2025-01-05,98220.51,98836.85,97276.79,98363.61,8095.637230,1736121599999,793931715.95463240,1685799,3877.90985000,380355340.46860360,0
...,...,...,...,...,...,...,...,...,...,...,...
2025-05-18,103126.65,106660.00,103105.09,106454.26,21599.987260,1747612799999,2262214972.07493840,3193300,11664.89904000,1221914552.05520120,0
2025-05-19,106454.27,107108.62,102000.00,105573.74,30260.035240,1747699199999,3150324504.59545150,4881371,14208.30900000,1480452895.08415620,0
2025-05-20,105573.73,107320.00,104184.72,106849.99,23705.482750,1747785599999,2509102444.66959820,3878351,12350.46275000,1307273652.38952320,0


In [18]:
depth = DataCollector.get_market_depth(symbol='BTCUSDT', limit=200)
depth

{'bids': [['109927.10000000', '3.64964000'],
  ['109927.09000000', '0.00005000'],
  ['109926.92000000', '0.00005000'],
  ['109926.77000000', '0.00015000'],
  ['109926.76000000', '0.00015000'],
  ['109926.75000000', '0.41462000'],
  ['109926.74000000', '0.61111000'],
  ['109926.56000000', '0.05412000'],
  ['109925.60000000', '0.03650000'],
  ['109924.45000000', '0.00277000'],
  ['109924.34000000', '0.00005000'],
  ['109923.22000000', '0.00005000'],
  ['109922.04000000', '0.00005000'],
  ['109922.02000000', '0.00010000'],
  ['109922.01000000', '1.90885000'],
  ['109922.00000000', '1.91455000'],
  ['109921.99000000', '0.14269000'],
  ['109921.76000000', '0.00005000'],
  ['109921.67000000', '0.05463000'],
  ['109921.60000000', '0.00025000'],
  ['109921.41000000', '0.00005000'],
  ['109921.39000000', '0.00005000'],
  ['109921.38000000', '0.58017000'],
  ['109921.37000000', '0.00010000'],
  ['109921.32000000', '0.04867000'],
  ['109920.61000000', '0.60020000'],
  ['109920.46000000', '0.00005